# Statistical and Multivariate Analysis
## Philippine Health Indicators

**Purpose**
Extract latent health patterns, confirm statistical relationships,
and segment health profiles using multivariate statistical techniques.

**Dataset Source**
https://www.kaggle.com/datasets/thedevastator/philippine-health-indicators

**Methods**
- Hypothesis testing
- Principal Component Analysis (PCA)
- Clustering (K-Means, Hierarchical)
- Regression analysis



In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage

import statsmodels.api as sm

sns.set(style="whitegrid")
pd.set_option("display.max_columns", 100)

# Load cleaned dataset
df = pd.read_csv("/content/cleaned_philippine_health_indicators.csv")

df.head()


In [ ]:
# Identify numeric variables for multivariate analysis
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Remove non-indicator numeric columns if present
numeric_cols = [c for c in numeric_cols if c not in ["Year"]]

numeric_cols


In [ ]:
# Perform ANOVA if Region column exists
if "Region" in df.columns:
    indicator = numeric_cols[0]

    groups = [
        df[df["Region"] == region][indicator].dropna()
        for region in df["Region"].unique()
    ]

    f_stat, p_value = stats.f_oneway(*groups)

    print(f"ANOVA for {indicator}")
    print(f"F-statistic: {f_stat:.3f}")
    print(f"P-value: {p_value:.5f}")


In [ ]:
# Example: Urban vs Rural
if "UrbanRural" in df.columns:
    indicator = numeric_cols[0]

    urban = df[df["UrbanRural"] == "Urban"][indicator].dropna()
    rural = df[df["UrbanRural"] == "Rural"][indicator].dropna()

    t_stat, p_value = stats.ttest_ind(urban, rural, equal_var=False)

    print(f"T-test for {indicator}")
    print(f"T-statistic: {t_stat:.3f}")
    print(f"P-value: {p_value:.5f}")


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[numeric_cols])


In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

explained_variance = pd.DataFrame({
    "Component": range(1, len(pca.explained_variance_ratio_) + 1),
    "Explained Variance Ratio": pca.explained_variance_ratio_,
    "Cumulative Variance": np.cumsum(pca.explained_variance_ratio_)
})

explained_variance


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(
    explained_variance["Component"],
    explained_variance["Explained Variance Ratio"],
    marker="o"
)
plt.xlabel("Principal Component")
plt.ylabel("Explained Variance Ratio")
plt.title("Scree Plot")
plt.show()


In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f"PC{i+1}" for i in range(pca.n_components_)],
    index=numeric_cols
)

loadings.iloc[:, :3]


In [ ]:
# Select number of clusters using silhouette score
silhouette_scores = {}

for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    silhouette_scores[k] = silhouette_score(X_scaled, labels)

silhouette_scores


In [ ]:
# Fit final KMeans
optimal_k = max(silhouette_scores, key=silhouette_scores.get)
kmeans = KMeans(n_clusters=optimal_k, random_state=42)

df["cluster_kmeans"] = kmeans.fit_predict(X_scaled)

df["cluster_kmeans"].value_counts()


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    hue=df["cluster_kmeans"],
    palette="tab10"
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-Means Clusters in PCA Space")
plt.show()


In [ ]:
linkage_matrix = linkage(X_scaled, method="ward")

plt.figure(figsize=(12, 6))
dendrogram(linkage_matrix, truncate_mode="level", p=5)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Sample Index")
plt.ylabel("Distance")
plt.show()


In [ ]:
# Define outcome and predictors
outcome = numeric_cols[0]
predictors = numeric_cols[1:5]

X = df[predictors]
y = df[outcome]

# Add constant
X = sm.add_constant(X)

model = sm.OLS(y, X, missing="drop").fit()
model.summary()


In [ ]:
# Identify dominant indicators per component
top_loadings = (
    loadings
    .abs()
    .sort_values(by="PC1", ascending=False)
    .head(5)
)

top_loadings


In [ ]:
# Attach PCA scores back to dataset
df["PC1"] = X_pca[:, 0]
df["PC2"] = X_pca[:, 1]

df[["PC1", "PC2"]].describe()


In [ ]:
explained_variance.to_csv(
    "/content/pca_explained_variance.csv",
    index=False
)

loadings.to_csv(
    "/content/pca_loadings.csv"
)

df.to_csv(
    "/content/multivariate_analysis_output.csv",
    index=False
)


## Key Findings from Statistical & Multivariate Analysis

- PCA reveals latent dimensions summarizing overall health status
- High-loading indicators represent structural health system factors
- Clustering identifies distinct health profiles across observations
- Regression confirms statistically significant associations between
  service coverage and health outcomes

These results support advanced modeling and policy stratification.

